# Math

We are going to implement all arithmetic operations here. These are the first opcodes that actually manipulate the stack and consume some gas.

Lets see how `add` works. We are going to `pop` 2 values from the stack add them and `push` back the result on the stack.

We also need to increment the `program counter (pc)` by one. In the end we need to deduct 3 gas for executing the `add` operation.

Most arithmetic opcodes work like this.

In [ ]:
pub fn add(evm: &mut State) {
    let a = evm.stack.pop();
    let b = evm.stack.pop();
    evm.stack.push(a + b);
    evm.pc += 1;
    evm.gas_dec(3);
}

In [ ]:
pub fn mul(evm: &mut State) {
    let a = evm.stack.pop();
    let b = evm.stack.pop();
    evm.stack.push(a * b);
    evm.pc += 1;
    evm.gas_dec(5);
}

In [ ]:
pub fn sub(evm: &mut State) {
    let a = evm.stack.pop();
    let b = evm.stack.pop();
    evm.stack.push(a - b);
    evm.pc += 1;
    evm.gas_dec(3);
}

One interesting note about how the EVM handles division by 0. Most other systems would throw an exception if you try to divide by 0. Not the EVM. It just returns 0.

Division by 0 are not directly handled by the EVM and are mostly a feature of the programming language like Solidity.

In [ ]:
pub fn div(evm: &mut State) {
    let a = evm.stack.pop();
    let b = evm.stack.pop();
    evm.stack.push(if b == 0 { 0 } else { a / b });
    evm.pc += 1;
    evm.gas_dec(5);
}

Exactly like div but we use the absolute value for the both the denominator and numerator.

Small little helper function to determine the sign of a number.

In [ ]:
let pos_or_neg = |number: i64| if number < 0 { -1 } else { 1 };

In [ ]:
pub fn sdiv(evm: &mut State) {
    let a = evm.stack.pop() as i64;
    let b = evm.stack.pop() as i64;

    let sign = if a * b < 0 { -1 } else { 1 }; // equivalent to pos_or_neg(a*b)

    evm.stack.push(if b == 0 { 0 } else { sign * (a.abs() / b.abs()) } as u64);
    evm.pc += 1;
    evm.gas_dec(5);
}

In [ ]:
pub fn modulo(evm: &mut State) {
    let a = evm.stack.pop();
    let b = evm.stack.pop();
    evm.stack.push(if b == 0 { 0 } else { a % b });
    evm.pc += 1;
    evm.gas_dec(5);
}

In [ ]:
pub fn smod(evm: &mut State) {
    let a = evm.stack.pop() as i64;
    let b = evm.stack.pop() as i64;

    let sign = if a * b < 0 { -1 } else { 1 }; // equivalent to pos_or_neg(a*b)

    evm.stack.push(if b == 0 { 0 } else { (a.abs() % b.abs()) * sign } as u64);
    evm.pc += 1;
    evm.gas_dec(5);
}

In [ ]:
pub fn addmod(evm: &mut State) {
    let a = evm.stack.pop();
    let b = evm.stack.pop();
    let n = evm.stack.pop();

    evm.stack.push((a + b) % n);
    evm.pc += 1;
    evm.gas_dec(8);
}

In [ ]:
pub fn mulmod(evm: &mut State) {
    let a = evm.stack.pop();
    let b = evm.stack.pop();
    let n = evm.stack.pop();

    evm.stack.push((a * b) % n);
    evm.pc += 1;
    evm.gas_dec(8);
}

The gas cost for exp is dynamic. It is a function of how many bytes we need to represent the exponent in binary. This helper function calculates this.

In [ ]:
pub fn size_in_bytes(number: i128) -> usize {
    if number == 0 {
        return 1;
    }
    let bits_needed = (number.abs() as f64 + 1.0).log2().ceil() as usize;
    (bits_needed + 7) / 8
}

In [ ]:
pub fn exp(evm: &mut Evm) {
    let a = evm.stack.pop();
    let exponent = evm.stack.pop();
    evm.stack.push(a.pow(exponent));
    evm.pc += 1;
    evm.gas_dec(10 + 50 * size_in_bytes(exponent));
}

More informations about this rarely used opcode `signextend` [here](https://ethereum.stackexchange.com/questions/63062/evm-signextend-opcode-explanation).

In [ ]:
pub fn signextend(evm: &mut Evm) {
    let b = evm.stack.pop();
    let x = evm.stack.pop();
    let result = if b <= 31 {
        let testbit = b * 8 + 7;
        let sign_bit = 1_u128 << testbit;
        if x & sign_bit != 0 {
            x | ((1_u128 << 256) - sign_bit)
        } else {
            x & (sign_bit - 1)
        }
    } else {
        x
    };

    evm.stack.push(result);
    evm.pc += 1;
    evm.gas_dec(5);
}